<a href="https://colab.research.google.com/github/naveenvarma600/SENTENCE_VIBES/blob/main/SentenceVibes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTING NECESSARY LIBRARIES

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json
import random

DEFINING NECESSARY CLASSES

In [3]:
class Sentiment:
  negative="NEGATIVE"
  positive="POSITIVE"
class Review:
  def __init__(self,text,score):
    self.text=text
    self.score=score
    self.sentiment=self.get_sentiment()
  def get_sentiment(self):
    if self.score<3:
      return Sentiment.negative
    else:#when score is 3 or 4 or 5
      return Sentiment.positive
class ReviewContainer:
  def __init__(self,reviews):
    self.reviews=reviews
  def evenly_distibute(self):
    negative=list(filter(lambda x:x.sentiment==Sentiment.negative,self.reviews))
    positive=list(filter(lambda x:x.sentiment==Sentiment.positive,self.reviews))
    #as data contains more positive data sets,we will reduce them into number of negative one's for better accuracy
    positive_shrunk=positive[:len(negative)]
    self.reviews=negative+positive_shrunk
    random.shuffle(self.reviews)

LOADING AND UNDERSTANDING DATA

In [4]:
reviews=[]
with open("Books_small_10000.json") as f:
  for line in f:
    review_json=json.loads(line)
    review=Review(review_json['reviewText'], review_json['overall'])
    reviews.append(review)

In [5]:
reviews[0].text
reviews[0].score
reviews[0].sentiment

'POSITIVE'

In [6]:
from sklearn.model_selection import train_test_split
training,testing=train_test_split(reviews,test_size=0.33)

In [7]:
len(training)

6700

In [8]:
#evenly distributing data for better accuracies
traincontainer=ReviewContainer(training)
traincontainer.evenly_distibute()
testcontainer=ReviewContainer(testing)
testcontainer.evenly_distibute()

In [9]:
#splitting data
x_train=[x.text for x in traincontainer.reviews]
x_test=[x.text for x in testcontainer.reviews]
y_train=[y.sentiment for y in traincontainer.reviews]
y_test=[y.sentiment for y in testcontainer.reviews]

In [10]:
y_train.count(Sentiment.positive)
y_train.count(Sentiment.negative)


422

In [11]:
#here we can also use count vectorizer but ,tfidf vectorizer best suits the problem
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer()
x_train_vector=vectorizer.fit_transform(x_train)
x_test_vector=vectorizer.transform(x_test)

SUPPORT VECTOR MACHINES

In [12]:
from sklearn.svm import SVC
clf_svm=SVC()
clf_svm.fit(x_train_vector,y_train)
y_pred=clf_svm.predict(x_test_vector)

from sklearn.metrics import accuracy_score,f1_score
clf_svm_score=accuracy_score(y_test,y_pred)
print(accuracy_score(y_test,y_pred))
print(f1_score(y_test,y_pred,average=None,labels=[Sentiment.positive,Sentiment.negative]))

0.831081081081081
[0.82837529 0.83370288]


DECISION TREES

In [13]:
from sklearn.tree import DecisionTreeClassifier
clf_dec=DecisionTreeClassifier()
clf_dec.fit(x_train_vector,y_train)
y_pred=clf_dec.predict(x_test_vector)

from sklearn.metrics import accuracy_score,f1_score
clf_dec_score=accuracy_score(y_test,y_pred)
print(accuracy_score(y_test,y_pred))
print(f1_score(y_test,y_pred,average=None,labels=[Sentiment.positive,Sentiment.negative]))

0.6373873873873874
[0.65524625 0.6175772 ]


LOGISTIC REGRESSION

In [14]:
from sklearn.linear_model import LogisticRegression
clf_log=LogisticRegression()
clf_log.fit(x_train_vector,y_train)
y_pred=clf_log.predict(x_test_vector)

from sklearn.metrics import accuracy_score,f1_score
clf_log_score=accuracy_score(y_test,y_pred)
print(accuracy_score(y_test,y_pred))
print(f1_score(y_test,y_pred,average=None,labels=[Sentiment.positive,Sentiment.negative]))

0.8333333333333334
[0.83408072 0.83257919]


from the above calculations,we can clearly say that logistic regression best suits the problem

In [17]:
#predicting few random samples whether positive or negative
clf_log.predict(vectorizer.transform(["very bad","i loved it","awesome","worst one"]))

array(['NEGATIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE'], dtype='<U8')